## Notes
##### ------------------------------------------------
* This code uses regridded datafiles.

##### ------------------------------------------------

### Set up workspace

In [1]:
import sys
sys.path.append('../')

In [2]:
import ESMF
import pickle
import numpy as np
import matplotlib.pyplot as plt
from cartopy.util import add_cyclic_point
import cartopy.crs as ccrs
from matplotlib import rc
%matplotlib inline

from plants_and_TCR.generate_figures import quick_map
from plants_and_TCR.analysis_parameters import directory_information
from plants_and_TCR.analysis_parameters import get_CMIP_info
from plants_and_TCR.process_data import make_processed_data_dict
from plants_and_TCR.process_data import regrid_and_process_datafiles
from plants_and_TCR.analyze_data import multi_model_stats

rc('font',**{'family':'sans-serif','sans-serif':['Arial']})

In [3]:
path_processed_datafiles = directory_information.DIR_PROCESSED_DATA
path_output = directory_information.DIR_OUTPUT_FIGURES

### Regrid files (only need to do once)

In [4]:
regrid_and_process_datafiles.process_data(varname='rsdscs')

regrid_and_process_datafiles.process_data(varname='rsds_clouds')

regrid_and_process_datafiles.process_data(varname='rldscs')

regrid_and_process_datafiles.process_data(varname='rlds_clouds')

regrid_and_process_datafiles.process_data(varname='rsus')

regrid_and_process_datafiles.process_data(varname='rlus')

regrid_and_process_datafiles.process_data(varname='hfls')

regrid_and_process_datafiles.process_data(varname='hfss')

regrid_and_process_datafiles.process_data(varname='LWnet')

regrid_and_process_datafiles.process_data(varname='SWnet')

regrid_and_process_datafiles.process_data(varname='SH+LH')

regrid_and_process_datafiles.process_data(varname='G')

regrid_and_process_datafiles.process_data(varname='Rnet_surf')

bcc-csm1-1_1pctCO2-rad_rsdscs not in the dictionary
rsdscs_1pctCO2-rad_CMIP5_bcc-csm1-1 is not in the dictionary
bcc-csm1-1_1pctCO2-bgc_rsdscs not in the dictionary
rsdscs_1pctCO2-bgc_CMIP5_bcc-csm1-1 is not in the dictionary
bcc-csm1-1_1pctCO2_rsdscs not in the dictionary
rsdscs_1pctCO2_CMIP5_bcc-csm1-1 is not in the dictionary
bcc-csm1-1_piControl_rsdscs not in the dictionary
rsdscs_piControl_CMIP5_bcc-csm1-1 is not in the dictionary
CanESM2_1pctCO2-rad_rsdscs not in the dictionary
rsdscs_1pctCO2-rad_CMIP5_CanESM2 is not in the dictionary
CanESM2_1pctCO2-bgc_rsdscs not in the dictionary
rsdscs_1pctCO2-bgc_CMIP5_CanESM2 is not in the dictionary
CanESM2_1pctCO2_rsdscs not in the dictionary
rsdscs_1pctCO2_CMIP5_CanESM2 is not in the dictionary
CanESM2_piControl_rsdscs not in the dictionary
rsdscs_piControl_CMIP5_CanESM2 is not in the dictionary
CESM1-BGC_1pctCO2-rad_rsdscs not in the dictionary
rsdscs_1pctCO2-rad_CMIP5_CESM1-BGC is not in the dictionary
CESM1-BGC_1pctCO2-bgc_rsdscs not 

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/disk/p/czarakas/anaconda3/envs/plants_and_TCR_v3/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3331, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-4-1382e69cc416>", line 9, in <module>
    regrid_and_process_datafiles.process_data(varname='rsus')
  File "../plants_and_TCR/process_data/regrid_and_process_datafiles.py", line 153, in process_data
    ds.to_netcdf(path=output_path+nametag+'.nc')
  File "/home/disk/p/czarakas/anaconda3/envs/plants_and_TCR_v3/lib/python3.7/site-packages/xarray/core/dataset.py", line 1545, in to_netcdf
    invalid_netcdf=invalid_netcdf,
  File "/home/disk/p/czarakas/anaconda3/envs/plants_and_TCR_v3/lib/python3.7/site-packages/xarray/backends/api.py", line 1097, in to_netcdf
    writes = writer.sync(compute=compute)
  File "/home/disk/p/czarakas/anaconda3/envs/plants_and_TCR_v3/lib/python3.7/site-packages/xarray/backends/common.py", line 204, in sy

KeyboardInterrupt: 

### Do analysis

In [ ]:
varlist=['rsdscs','rsds_clouds','rsus','SWnet',
         'rldscs','rlds_clouds','rlus','LWnet',
         'hfls','hfss','G','Rnet_surf']
varlist_full=['SW Down, Clearsky','SW Down, Clouds','SW Up','Net SW',
         'LW Down, Clearsky','LW Down, Clouds','LW Up','Net LW',
         'LH','SH','Heat Storage','Net Radiation']

RUNNAME_INDS=[2,0] #[1,3] #[2,0]
END_YR=70
CMIP_NAMES=['CMIP6']

In [ ]:
def plot_subplot(mapdata, lat, lon, clim, cmap, clevels=None, sighatch=False, sigmask=None, p=0.0,
                markerstyle='x', markersize=0.2, extend_choice='both'):
    ax.coastlines()
    ax.set_global()
    cyclic_data, cyclic_lons = add_cyclic_point(mapdata, coord=lon)

    # plot our data:
    if clevels is not None:
        clevels=clevels
    else:
        clevels= np.arange(clim[0],clim[1],color_interval)
    #cs = plt.pcolormesh(cyclic_lons, lat, cyclic_data, transform=ccrs.PlateCarree())
    cs = plt.contourf(cyclic_lons, lat, cyclic_data, clevels, cmap = cmap, extend=extend_choice,
                              transform=ccrs.PlateCarree())#, vmin=-1.5, vmax=1.5)
    cs.set_clim(clim)

    # Choose your colormap
    plt.set_cmap(cmap)

    plt.clim(clim)
    
    if sighatch:
        cyclic_sig, cyclic_lons = add_cyclic_point(sigmask, coord=lon)

        CLN, CLT = np.meshgrid(cyclic_lons, sigmask['lat'])

        # only put the hatches where the sigmask is < p 
        lat_sig = np.where(cyclic_sig < p, CLT, np.nan)
        lon_sig = np.where(cyclic_sig < p, CLN, np.nan)
        
        hatch = ax.scatter(lon_sig,
                           lat_sig,
                           marker=markerstyle,
                           s=markersize,
                           c=[0.6, 0.6, 0.6],
                           alpha=0.8,
                           transform=ccrs.PlateCarree())
    
    ax.patch.set_alpha(1.0)

In [ ]:
fig = plt.figure(figsize=(40, 17))
fontsize = 35
clim=[-3,3]
clevs = np.round(np.arange(-3.3,3.3,0.3),1)
cbar_inds = np.round(np.arange(-3,3.3,0.6),1)
cmap=plt.cm.PuOr

for i in range(0, len(varlist)):
    #if i!=10:
    var=varlist[i]
    print('------------------'+var+'------------------')
    PROC_DATA_DICT = make_processed_data_dict.create_variable_dictionary(runnames=['1pctCO2', '1pctCO2-rad'],
                                                                         varname=var,
                                                                         input_path=path_processed_datafiles,
                                                                         cdict_names=['CMIP5','CMIP6'])


    [ds_all_models_phys,
     multimodelSum_phys,
     positiveChangeCount_phys,
     negativeChangeCount_phys,
     num_models_with_data_phys] = multi_model_stats.get_mm_mean(proc_data_dict=PROC_DATA_DICT,
                                                                varname=var,
                                                                end_yr=END_YR,
                                                                change_cutoff=0,
                                                                runname_inds=RUNNAME_INDS,
                                                                cmip_names = CMIP_NAMES)

    mostlyAgree = multi_model_stats.calculate_where_models_mostly_agree(positiveChangeCount_phys,
                                                                    negativeChangeCount_phys,
                                                                    cut_off_num = 9,
                                                                    num_models=num_models_with_data_phys)

    ######################## Make map for subplot ##########################
    ax=plt.subplot(3, 4, i+1, projection=ccrs.Robinson())
    plt.title(varlist_full[i], fontsize=fontsize, y=1)
    mapdata = ds_all_models_phys[var].mean(dim='modelname', skipna=True).values#*(ds_all_models_phys['da_land'].values/100)*(1-ds_all_models_phys['da_glac'].values/100)
    lat = ds_all_models_phys['lat'].values
    lon = ds_all_models_phys['lon'].values
    plot_subplot(mapdata, lat, lon, clim=clim, clevels = clevs, cmap=cmap, 
                 sighatch=True, sigmask=-mostlyAgree)
    
################ Full figure formatting ##########################################

# adjust the subplots to make space for a colorbar and to decrease spacing between subplots
plt.subplots_adjust(bottom=0.08, right=None, top=None, wspace=.02, hspace=.15)

# make an axes for the colorbar
cax = plt.axes([0.25, 0.05, 0.5, 0.018], frameon='no')

# add a colorbar:
#cbar = plt.colorbar(cax=cax,orientation='horizontal', extend='both')
cbar = plt.colorbar(cax=cax, orientation='horizontal', extend='both', ticks=cbar_inds)
cbar.ax.tick_params(labelsize=fontsize)

# set color limits for the colorbar
cbar.set_clim(clim)

# put a label on the colorbar, e.g. with the units of your field
cbar.set_label('$W/m^2$', fontsize=fontsize)
fig.savefig(path_output +'MultiModelMean_SurfEnergyBudget_CMIP6_nosighatch' +'.png', bbox_inches='tight')
fig.savefig(path_output +'MultiModelMean_SurfEnergyBudget_CMIP6_nosighatch' +'.eps', rasterized=False) # 3.8M!